In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ========================= Q1: Perform Exploratory Data Analysis (EDA) =========================
# Load the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Print dataset shape
print(f"Training Data Shape: {x_train.shape}, Labels: {y_train.shape}")
print(f"Test Data Shape: {x_test.shape}, Labels: {y_test.shape}")

# Display 9 sample images with their labels
plt.figure(figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_train[i], cmap='gray')  # Show images in grayscale
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.show()

# Plot label distribution
unique, counts = np.unique(y_train, return_counts=True)
plt.figure(figsize=(8, 5))
sns.barplot(x=unique, y=counts, palette="viridis")
plt.xlabel("Digit Label")
plt.ylabel("Count")
plt.title("Distribution of Digit Labels in Training Data")
plt.show()

# Check pixel value statistics
print(f"Pixel Value Range: Min = {x_train.min()}, Max = {x_train.max()}")
print(f"Mean Pixel Value: {x_train.mean():.2f}, Standard Deviation: {x_train.std():.2f}")

# ========================= Q1: Train Simple ANN =========================
# Normalize the dataset (scale pixel values to range [0,1])
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images from 28x28 to 784
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Define a basic ANN model
def create_basic_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train basic ANN model
model_basic = create_basic_model()
history_basic = model_basic.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Evaluate basic model
basic_loss, basic_acc = model_basic.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy (Basic ANN): {basic_acc:.4f}")

# ========================= Q2(A): Compare Accuracy With and Without Dropout =========================
def create_dropout_model():
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train model WITHOUT dropout
model_no_dropout = create_dropout_model()
history_no_dropout = model_no_dropout.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

# Train model WITH dropout
model_with_dropout = create_dropout_model()
history_with_dropout = model_with_dropout.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

# Evaluate models
loss_no_dropout, acc_no_dropout = model_no_dropout.evaluate(x_test, y_test, verbose=2)
loss_with_dropout, acc_with_dropout = model_with_dropout.evaluate(x_test, y_test, verbose=2)

print(f"Test Accuracy WITHOUT Dropout: {acc_no_dropout:.4f}")
print(f"Test Accuracy WITH Dropout (0.3): {acc_with_dropout:.4f}")

# Plot accuracy & loss comparison
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_no_dropout.history['val_accuracy'], label='No Dropout')
plt.plot(history_with_dropout.history['val_accuracy'], label='With Dropout')
plt.xlabel('Epochs')
plt.ylabel('Test Accuracy')
plt.legend()
plt.title('Test Accuracy Comparison')

plt.subplot(1, 2, 2)
plt.plot(history_no_dropout.history['val_loss'], label='No Dropout')
plt.plot(history_with_dropout.history['val_loss'], label='With Dropout')
plt.xlabel('Epochs')
plt.ylabel('Test Loss')
plt.legend()
plt.title('Test Loss Comparison')

plt.show()

# ========================= Q2(B): Compute Classification Metrics & Save/Reload Model =========================
# Make predictions on test data
y_pred = model_with_dropout.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

# Compute classification metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print the metrics
print("="*40)
print("|   Metric      |   Value    |")
print("="*40)
print(f"| Accuracy      |   {accuracy:.4f}   |")
print(f"| Precision     |   {precision:.4f}   |")
print(f"| Recall        |   {recall:.4f}   |")
print(f"| F1 Score      |   {f1:.4f}   |")
print("="*40)

# Save model
model_path = "fashion_mnist_ann_model.h5"
model_with_dropout.save(model_path)
print(f"Model saved at {model_path}")

# Reload model
loaded_model = keras.models.load_model(model_path)
print("Model reloaded successfully!")

# Evaluate reloaded model
reloaded_loss, reloaded_acc = loaded_model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy (Reloaded Model): {reloaded_acc:.4f}")

Basic ANN Accuracy: 87.98%

With Dropout (0.3) Accuracy: 88.80% (almost same as without dropout)

Classification Metrics:

Accuracy: 88.80%

Precision: 88.87%

Recall: 88.80%

F1 Score: 88.82%

Model saved as fashion_mnist_ann_model.h5 and successfully reloaded.

Reloaded model accuracy matches the trained model (88.80%), confirming successful saving & loading.

-- Code Explaination

Exploratory Data Analysis (EDA)

The dataset consists of 60,000 training and 10,000 test images, each of 28×28 pixels.

The images belong to 10 different clothing categories, and the label distribution is fairly uniform.

The pixel values range from 0 to 255, with a mean of ~72 and standard deviation of ~90.

Basic ANN Model Performance

A simple 2-layer neural network (128 neurons, ReLU activation, softmax output) was trained.

Achieved ~87.98% test accuracy after 10 epochs.

Comparison of Accuracy with and without Dropout

Added Dropout layers (30%) to improve generalization.

Without Dropout: 88.87% test accuracy

With Dropout (0.3): 88.80% test accuracy

No significant performance drop, indicating the model generalizes well.

Evaluation Metrics

Accuracy: 0.8880

Precision: 0.8887

Recall: 0.8880

F1 Score: 0.8882

These values show that the model is well-balanced in performance.